# Exercise 2.5

### 1. Import all Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from keplergl import KeplerGl


In [2]:
df = pd.read_csv("citibike_weather_2022.csv", low_memory=False)


### 2. Create a new column with value 1

In [3]:
df["trip_count"] = 1


#Create the aggregated dataframe

agg_df = (
    df.groupby(["start_station_name", "end_station_name"])["trip_count"]
      .sum()
      .reset_index()
)
agg_df.head()


,start_station_name,end_station_name,trip_count
0,1 Ave & E 110 St,1 Ave & E 110 St,27
1,1 Ave & E 110 St,1 Ave & E 44 St,2
2,1 Ave & E 110 St,1 Ave & E 68 St,1
3,1 Ave & E 110 St,1 Ave & E 78 St,3
4,1 Ave & E 110 St,1 Ave & E 94 St,15


### 3.Initialize a kepler.gl map instance

In [4]:
import ipywidgets as widgets
widgets.IntSlider()


IntSlider(value=0)

In [5]:
from keplergl import KeplerGl

map_1 = KeplerGl()

map_1 = KeplerGl(height=600)


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


### 4.Add data to the kepler map

In [7]:
# 1) Make station lookup tables (start + end) with coordinates
start_coords = (
    df[["start_station_name", "start_lat", "start_lng"]]
    .dropna()
    .drop_duplicates("start_station_name")
)

end_coords = (
    df[["end_station_name", "end_lat", "end_lng"]]
    .dropna()
    .drop_duplicates("end_station_name")
)

# 2) Merge coordinates into the aggregated trips table
agg_map = (
    agg_df
    .merge(start_coords, on="start_station_name", how="left")
    .merge(end_coords, on="end_station_name", how="left")
    .dropna(subset=["start_lat", "start_lng", "end_lat", "end_lng"])
)

# 3) Load into kepler.gl
from keplergl import KeplerGl
map_1 = KeplerGl(height=650, data={"Trips (station-to-station)": agg_map})

map_1

# Save the interactive Kepler map to an HTML file
map_1.save_to_html(file_name="kepler_map.html")



User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to kepler_map.html!


In [8]:
map_1 = KeplerGl(height=650, data={"Trips (station-to-station)": agg_map})
map_1.save_to_html(file_name="kepler_map.html")
print("Saved: kepler_map.html (open it in your browser)")


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to kepler_map.html!
Saved: kepler_map.html (open it in your browser)


I customized the Kepler map by creating a Point layer using the start station coordinates and selecting a color palette (e.g., Blues/Viridis) so station locations are easy to interpret without overpowering the basemap. I then added an Arc layer using the start and end coordinates to visualize flows between stations. I chose a contrasting warm palette (e.g., Reds/Plasma) for arcs so trip connections stand out from the points, and I mapped arc thickness and point size to trip_count to emphasize the most frequent station-to-station routes.

### 5.What makes an impression

When you filter to only the highest trip_count trips, the densest arcs/points typically cluster around major transit hubs and job/activity centers—especially in Midtown and Lower Manhattan. Prior research on Citi Bike use has found that the busiest stations tend to be near major hubs like Grand Central Terminal, Penn Station, and Port Authority, reflecting commute and last-mile connections.

After filtering the map to show the most common trips, the visualization reveals a strong concentration of routes in Manhattan, particularly around Midtown and Lower Manhattan. These areas appear especially busy, with many high-frequency trips connecting nearby stations. This pattern suggests that Citi Bike is heavily used for short-distance travel and commuting in dense commercial and transit-oriented zones, which aligns with prior findings that usage is highest near employment centers and transportation hubs in New York City.

I also notice strong clusters in high-demand neighborhoods like parts of Midtown/Downtown Manhattan and areas of Brooklyn (e.g., Downtown Brooklyn/Williamsburg), which aligns with city statements that these are among the system’s busiest existing areas.

### 6.Create the config object

In [9]:
config = map_1.config


In [12]:
#Save to HTML 

map_1.save_to_html(
    file_name="kepler_trips_configured.html",
    config=config
)


Map saved to kepler_trips_configured.html!


In [11]:
#Save as JSON

import json

with open("kepler_config.json", "w") as f:
    json.dump(config, f)
